In [1]:
import numpy as np
from data_manager import DataManager
from schema import Schema

ERROR:Schema:Failed to import a package! Be wary about continuing...
Traceback (most recent call last):
  File "c:\Users\Sam Wolk\Dropbox\mit\2023_spring\ml\ml-for-building-energy-modeling\ml-for-bem\schema.py", line 25, in <module>
    from pyumi.shoeboxer.shoebox import ShoeBox
ModuleNotFoundError: No module named 'pyumi'


In [2]:
schema = Schema()
data_manager = DataManager(schema)

INFO:Model:Full monthly data set not found.  Downloading from GCloud Bucket...
INFO:Storage:Downloading bucket:all_data_monthly.hdf5 to file:c:\Users\Sam Wolk\Dropbox\mit\2023_spring\ml\ml-for-building-energy-modeling\ml-for-bem\data\model_data_manager\all_data_monthly.hdf5...
INFO:Storage:Done downloading.
INFO:Model:Done downloading dataset!
INFO:Storage:Downloading bucket:default_schedules.npy to file:c:\Users\Sam Wolk\Dropbox\mit\2023_spring\ml\ml-for-building-energy-modeling\ml-for-bem\data\model_data_manager\default_schedules.npy...
INFO:Storage:Done downloading.
INFO:Model:Loading the full dataset into main RAM...
INFO:Model:Finished loading the full dataset.
INFO:Model:Full Input Batch Size (in storage form, not MLVec Form): 382.750176MB
INFO:Model:Loading climate data...
INFO:Storage:Downloading bucket:tsol.npy to file:c:\Users\Sam Wolk\Dropbox\mit\2023_spring\ml\ml-for-building-energy-modeling\ml-for-bem\data\epws\tsol.npy...
INFO:Storage:Done downloading.
INFO:Storage:Downlo

The following cell can be used to geneate cached training data

In [13]:
# uncomment to handle generating cached permutations
train_batch = data_manager.full_storage_batch[0:100000] 
targets = data_manager.results["total_heating"][0:100000]
data_manager.make_batch_permutations(train_batch, targets, 1, "training_demo")

INFO:Model:Making schedules... this may take a while...
INFO:Model:Making dataset permutations...
INFO:Model:Making permutation 0...
INFO:Model:Saving permutation 0...


Minibatches can be used when dealing with a very large training dataset which can't be loaded into memory all at once.

In [3]:
minibatch_size = 10000

In [15]:
mb = data_manager.load_minibatch("training_demo",0,919, minibatch_size)
mb.timeseries.shape, mb.ml_vectors.shape

((10000, 11, 8760), (10000, 62))

You can also more manually construct the minibatches, but it comes at a time price.  However, if you can fit everything into memory, do this once and be done with it. 

In [11]:
minibatch_size = 10000

In [12]:
ixs = np.random.choice(np.arange(500000, dtype=int), minibatch_size, replace=False)
minibatch = data_manager.full_storage_batch[ixs]
climate_timeseries = data_manager.get_batch_climate_timeseries(minibatch)
bldg_params = data_manager.get_building_vector(minibatch)
schedules = data_manager.make_schedules(minibatch)
timeseries = np.concatenate((climate_timeseries, schedules), axis=1)

# TODO: use schema based normalization, or just defining globals, make sure to avoid normalizing one hots.
# TODO: make sure windows, facademass, get set to correct values due to archetypal autoadjustment before normalizing

In [9]:
print(schema)

-------- Schema --------
---- batch_id ----
shape storage: (1,) / shape ml: (0,)
location storage: 0->1 / location ml: 0->0

---- variation_id ----
shape storage: (1,) / shape ml: (0,)
location storage: 1->2 / location ml: 0->0

---- program_type ----
shape storage: (1,) / shape ml: (19,)
location storage: 2->3 / location ml: 0->19

---- vintage ----
shape storage: (1,) / shape ml: (1,)
location storage: 3->4 / location ml: 19->20

---- climate_zone ----
shape storage: (1,) / shape ml: (17,)
location storage: 4->5 / location ml: 20->37

---- base_epw ----
shape storage: (1,) / shape ml: (0,)
location storage: 5->6 / location ml: 37->37

---- width ----
shape storage: (1,) / shape ml: (1,)
location storage: 6->7 / location ml: 37->38

---- height ----
shape storage: (1,) / shape ml: (1,)
location storage: 7->8 / location ml: 38->39

---- facade_2_footprint ----
shape storage: (1,) / shape ml: (1,)
location storage: 8->9 / location ml: 39->40

---- perim_2_footprint ----
shape storage: (

In [10]:
param_ix = 56

print(bldg_params.shape)
print(np.min(bldg_params, axis=0), np.max(bldg_params, axis=0))
np.min(bldg_params[:,param_ix]), np.max(bldg_params[:,param_ix])

(5000, 62)
[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.00522152 0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.4787234  0.69148936] [1.         1.         1.         1.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         1.         0.         0.         1.         0.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         0.         1.
 1.         0.6037847

(0.0, 0.5327971222760083)